Programa utilizado para la extracción de datos.

En este notebook se recoge la metodología utilizada para extraer toda la información turística y recopilarla en diferentes archivos de formato .csv.

Para ello, se utilzia la librería pytesseract, es una biblioteca de Python que actúa como una envoltura (wrapper) para el motor de Reconocimiento Óptico de Caracteres (OCR) de código abierto de Google, llamado Tesseract:

https://pypi.org/project/pytesseract/

De esta forma podemos extraer los datos de las imagenes y guardarlos en csv.

In [2]:
# Carga de librerías
import os
import pytesseract
from PIL import Image
import pandas as pd
import re


Por comodidad dejo la función completa en este ipynb. en vez de guardarlo en el script main.py.

Las funciones guardadas en el main.py serán exclusivamente para la limpieza y visualización de datos.

In [6]:

BASE_PATH = "../data/Tourism_jpgs"


number_regex = r"[-+]?\d[\d,.]*"
MONTHS = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]


# ============================================================
#  FUNCIÓN OCR
# ============================================================
def get_ocr_numbers(img_path):
    text = pytesseract.image_to_string(Image.open(img_path))
    lines = [l.strip() for l in text.split("\n") if l.strip()]
    
    rows = []
    for line in lines:
        nums = re.findall(number_regex, line)
        if len(nums) >= 1:
            rows.append(nums)
    return rows



# ============================================================
#  RECORRER CADA CARPETA POR SEPARADO
# ============================================================
for folder in sorted(os.listdir(BASE_PATH)):

    folder_path = os.path.join(BASE_PATH, folder)
    if not os.path.isdir(folder_path):
        continue

    print(f"\nProcesando carpeta: {folder}")

    # Nombres de países (orden correcto)
    countries = [c.capitalize() for c in folder.split("_")]
    expected_cols = len(countries)

    folder_rows = []   # datos SOLO para esta carpeta


    # ========================================================
    #  PROCESAR TODAS LAS IMÁGENES DE ESTA CARPETA
    # ========================================================
    for file in sorted(os.listdir(folder_path)):
        if not file.endswith(".png"):
            continue
        
        year = file.replace(".png", "")
        img_path = os.path.join(folder_path, file)

        # extraer OCR:
        ocr_rows = get_ocr_numbers(img_path)
        ocr_rows = ocr_rows[:12]  # los 12 meses reales

        # procesar cada mes
        for i, nums in enumerate(ocr_rows):
            month = MONTHS[i]

            # corregir desplazamientos:
            if len(nums) < expected_cols:
                nums = [""] * (expected_cols - len(nums)) + nums
            nums = nums[-expected_cols:]

            row = {"Year": int(year), "Month": month}

            for j, country in enumerate(countries):
                row[country] = nums[j]

            folder_rows.append(row)



    # ========================================================
    #  CREAR DATAFRAME DE ESTA CARPETA
    # ========================================================
    df = pd.DataFrame(folder_rows)

    # limpiar números
    for col in df.columns:
        if col not in ["Year", "Month"]:
            df[col] = df[col].astype(str).str.replace(",", "", regex=False)
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # ========================================================
    #  GUARDAR CSV INDIVIDUAL (MISMO NOMBRE SIEMPRE)
    # ========================================================
    output_name = f"{folder}.csv"
    df.to_csv(output_name, index=False)
    print(f"Guardado → {output_name}")



Procesando carpeta: Argentina_Australia_Austria_Belgium
Guardado → Argentina_Australia_Austria_Belgium.csv

Procesando carpeta: Canada_Colombia_Korea_Philippines
Guardado → Canada_Colombia_Korea_Philippines.csv

Procesando carpeta: DominicanRep_Singapore_SouthAfrica_UK
Guardado → DominicanRep_Singapore_SouthAfrica_UK.csv

Procesando carpeta: Finland_Hungary_India_Malaysia
Guardado → Finland_Hungary_India_Malaysia.csv

Procesando carpeta: France_Greece_Italy
Guardado → France_Greece_Italy.csv

Procesando carpeta: Netherlands_NewZealand_Norway_Portugal
Guardado → Netherlands_NewZealand_Norway_Portugal.csv

Procesando carpeta: Thailand_Tunisia_Turkiye
Guardado → Thailand_Tunisia_Turkiye.csv

Procesando carpeta: albania_egypt_morocco_switz
Guardado → albania_egypt_morocco_switz.csv

Procesando carpeta: brazil_peru_poland_usa
Guardado → brazil_peru_poland_usa.csv

Procesando carpeta: bulgaria_indonesia_sweden_uzbek
Guardado → bulgaria_indonesia_sweden_uzbek.csv

Procesando carpeta: chile_i

Una vez generados los csv se guardan en sus respectivas carpetas dentro de data<Tourism_jpgs